# Chapter 3

## Imports

In [1]:
import requests
import pprint
import pandas as pd

## Exploring an API to find and collect temperature data

In [2]:
def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the
    weather API passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to
                    make a GET request to.
        - payload:  A dictionary of data to pass along
                    with the request.
    
    returns:
        A response object.
    """
    return requests.get(
        'https://www.ncdc.noaa.gov/cdo-web/'
        f'api/v2/{endpoint}',
        headers={'token': 'jzZnLpQjvZhVYseBfrCwSXgoIXTCyTbE'},
        params=payload
    )

In [3]:
response = make_request(
    'datasets',
    {'startdate': '2018-10-01'}
)

In [4]:
response.status_code

200

In [5]:
response.ok

True

In [6]:
payload=response.json()

In [7]:
payload.keys()

dict_keys(['metadata', 'results'])

In [8]:
payload['metadata']

{'resultset': {'offset': 1, 'count': 11, 'limit': 25}}

In [9]:
payload['results'][0].keys()

dict_keys(['uid', 'mindate', 'maxdate', 'name', 'datacoverage', 'id'])

In [10]:
[(data['id'], data['name']) for data in payload['results']]

[('GHCND', 'Daily Summaries'),
 ('GSOM', 'Global Summary of the Month'),
 ('GSOY', 'Global Summary of the Year'),
 ('NEXRAD2', 'Weather Radar (Level II)'),
 ('NEXRAD3', 'Weather Radar (Level III)'),
 ('NORMAL_ANN', 'Normals Annual/Seasonal'),
 ('NORMAL_DLY', 'Normals Daily'),
 ('NORMAL_HLY', 'Normals Hourly'),
 ('NORMAL_MLY', 'Normals Monthly'),
 ('PRECIP_15', 'Precipitation 15 Minute'),
 ('PRECIP_HLY', 'Precipitation Hourly')]

In [11]:
response = make_request(
    'datacategories', payload={'datasetid': 'GHCND'}
)

In [12]:
response.status_code

200

In [13]:
response.json()['results']

[{'name': 'Evaporation', 'id': 'EVAP'},
 {'name': 'Land', 'id': 'LAND'},
 {'name': 'Precipitation', 'id': 'PRCP'},
 {'name': 'Sky cover & clouds', 'id': 'SKY'},
 {'name': 'Sunshine', 'id': 'SUN'},
 {'name': 'Air Temperature', 'id': 'TEMP'},
 {'name': 'Water', 'id': 'WATER'},
 {'name': 'Wind', 'id': 'WIND'},
 {'name': 'Weather Type', 'id': 'WXTYPE'}]

In [14]:
response = make_request(
    'datatypes',
    payload={'datacategoryid': 'TEMP', 'limit': 100}
)

In [15]:
response.status_code

200

In [16]:
[(datatype['id'], datatype['name'])
for datatype in response.json()['results']]

[('CDSD', 'Cooling Degree Days Season to Date'),
 ('DATN',
  'Number of days included in the multiday minimum temperature (MDTN)'),
 ('DATX',
  'Number of days included in the multiday maximum temperature (MDTX)'),
 ('DLY-DUTR-NORMAL', 'Long-term averages of daily diurnal temperature range'),
 ('DLY-DUTR-STDDEV',
  'Long-term standard deviations of daily diurnal temperature range'),
 ('DLY-TAVG-NORMAL', 'Long-term averages of daily average temperature'),
 ('DLY-TAVG-STDDEV',
  'Long-term standard deviations of daily average temperature'),
 ('DLY-TMAX-NORMAL', 'Long-term averages of daily maximum temperature'),
 ('DLY-TMAX-STDDEV',
  'Long-term standard deviations of daily maximum temperature'),
 ('DLY-TMIN-NORMAL', 'Long-term averages of daily minimum temperature'),
 ('DLY-TMIN-STDDEV',
  'Long-term standard deviations of daily minimum temperature'),
 ('EMNT', 'Extreme minimum temperature for the period.'),
 ('EMXT', 'Extreme maximum temperature for the period.'),
 ('HDSD', 'Heating De

In [17]:
response = make_request(
    'locationcategories', payload={'datasetid': 'GHCND'}
)

In [18]:
response.status_code

200

In [19]:
pprint.pprint(response.json())

{'metadata': {'resultset': {'count': 12, 'limit': 25, 'offset': 1}},
 'results': [{'id': 'CITY', 'name': 'City'},
             {'id': 'CLIM_DIV', 'name': 'Climate Division'},
             {'id': 'CLIM_REG', 'name': 'Climate Region'},
             {'id': 'CNTRY', 'name': 'Country'},
             {'id': 'CNTY', 'name': 'County'},
             {'id': 'HYD_ACC', 'name': 'Hydrologic Accounting Unit'},
             {'id': 'HYD_CAT', 'name': 'Hydrologic Cataloging Unit'},
             {'id': 'HYD_REG', 'name': 'Hydrologic Region'},
             {'id': 'HYD_SUB', 'name': 'Hydrologic Subregion'},
             {'id': 'ST', 'name': 'State'},
             {'id': 'US_TERR', 'name': 'US Territory'},
             {'id': 'ZIP', 'name': 'Zip Code'}]}


In [22]:
def get_item(name, what, endpoint, start=1, end=None):
    """
    Grab the JSON payload using binary search.
    
    Parameters:
        - name: The item to look for.
        - what: Dictionary specifying what item `name` is.
        - endpoint: Where to look for the item.
        - start: The position to start at.  We don't need
          to touch this, but the function will manipulate
          this with recursion.
        - end: The last position of the items. Used to
          find the midpoint, but like `start` this is not
          something we need to worry about.
          
    Returns: Dictionary of the information for the item
             if found, otherwise an empty dictionary.
    """
    # find the midpoint to cut the data in half each time
    mid = (start + (end or 1)) // 2
    
    # lowercase the name so this is not case-sensitive
    name = name.lower()
    
    # define the payload wewill send with each request
    payload = {
        'datasetid': 'GHCND', 'sortfield': 'name',
        'offset': mid, # we'll change the offset each time
        'limit': 1 # we only want one value back
    }
    
    # make request adding additional filters from `what`
    response = make_request(endpoint, {**payload, **what})
    
    if response.ok:
        payload = response.json()
        
        # if ok, grab the end index from the response
        # metadata the first time through
        end = end or \
            payload['metadata']['resultset']['count']
        
        # grab the lowercase version of the current name
        current_name = \
            payload['results'][0]['name'].lower()
        
        # if what we are searching for is in the current
        # name, we have found our item
        if name in current_name:
            # return the found item
            return payload['results'][0]
        else:
            if start >= end:
                # if start index is greater than or equal
                # to end index, we couldn't find it
                return {}
            elif name < current_name:
                # name comes before the current name in the
                # alphabet => search further to the left
                return get_item(name, what, endpoint,
                               start, mid - 1)
            elif name > current_name:
                # name comes after the current name in the
                # alphabet => searh further to the right
                return get_item(name, what, endpoint, 
                               mid + 1, end)
    else:
        # response wasn't ok, use code to determine why
        print('Response not OK, '
             f'status: {response.status_code}')

In [23]:
nyc = get_item('New York',
               {'locationcategoryid': 'CITY'},
               'locations'
              )

In [ ]:
nyc

In [25]:
central_park = get_item('NY City Central Park',
                       {'locationid': nyc['id']},
                       'stations'
                       )

In [ ]:
central_park

In [27]:
response = make_request('data',
                       {'datasetid': 'GHCND',
                       'stationid': central_park['id'],
                       'locationid': nyc['id'],
                       'startdate': '2018-10-01',
                       'enddate': '2018-10-31',
                       'datatypeid': ['TAVG', 'TMAX', 'TMIN'],
                       'units': 'metric',
                       'limit': 1000}
                       )

In [ ]:
response.status_code

In [ ]:
df = pd.DataFrame(response.json()['results'])
df.head()

In [ ]:
df.datatype.unique()

In [ ]:
if get_item('NY City Central Park',
           {'locationid': nyc['id'],
           'datatypeid': 'TAVG'},
           'stations'
           ):
    print('Found!')